# Process results

This notebook combines the various collected results into a single CSV that can be easily read using Pandas.

### Imports

In [2]:
#Thesis libs
from JQCLib.Preprocessing.Utils import readWebCircuits, saveMetrics, loadMetrics, loadMoreMetrics
from JQCLib.Metrics.MetricsWrapperClass import MetricWrapper
from JQCLib.Preprocessing.Utils import ToQiskit
from JQCLib.Metrics.DKLFPMetrics import DKLFP

#Other libs
from progressbar import ProgressBar, Bar, Percentage
import numpy as np
import pandas as pd
from qiskit.circuit.library import RealAmplitudes
from scipy import stats

### Files & folders

In [4]:
#Files in thesis
circuitFile =   'JQCLib\\Results\\trainingresults.txt'
metricsFile =   'JQCLib\\Results\\metrics.pkl'
fidelities =    'JQCLib\\Results\\Fidelities'
DATAFILE =      'JQCLib\\Results\\PQC_Dataset.csv'

#Load files
models = readWebCircuits(circuitFile)
metricDict = loadMetrics(metricsFile)
fidelities = loadMoreMetrics(fidelities)

fidelities = dict(sorted(fidelities.items()))
print("Found",len(fidelities),"results")

Found 10000 results


### Calculate metrics

In [5]:
#Main loop

pbar = ProgressBar(widgets=[Percentage(), Bar()], maxval=len(models)).start()
i = 0
for Id, TrainResult in models.items(): 
    
    #Check if metrics for the given circuit is missing
    if not Id in metricDict:

        #Create new entry           
        metricDict[Id] = MetricWrapper({})
    
    #Entry is now present. Either from file or newly inserted
    #Check if the entry is complete according to MetricWrapper rules    
    metricDict[Id].Validate(TrainResult)
    
    i+=1
    if(i%1000 == 0):
        saveMetrics(metricsFile,metricDict)    
    pbar.update(i)

pbar.finish()

print("Saving file.")
saveMetrics(metricsFile,metricDict)

100%|#########################################################################|


Saving file.


### Compute Frame Potentials & Divergence

In [10]:
circCount = 0
BINS = 1000
moreMetrics = {}

for Id, TrainResult in models.items():    
    
    #Check if 'moreMetrics' for the given circuit is missing
    if not Id in moreMetrics:

        #Create new entry           
        moreMetrics[Id] = DKLFP(fidelities[Id],BINS,4)
        circCount +=1 

print("Computed DKLFP for",circCount,"circuits")

Computed DKLFP for 10000 circuits


### Combine into CSV file

In [11]:
moreMetricLabels = ["FP1","FP2","DKL"]

#Metrics
data = np.array([list(wrapper.Metrics.values()) for wrapper in metricDict.values()])

#MoreMetrics
data = np.append(data,np.array(list(moreMetrics.values())),axis=1)

#Id
data = np.insert(data, 0, np.array(list(metricDict.keys())), axis=1)

#Metric labels
labels = ["Id"] + list(MetricWrapper.MustContain.keys()) + moreMetricLabels

#Create dataframe
df = pd.DataFrame(data, columns=labels)

#Cast applicable columns to INT and set Index as the MySql index
df = df.astype({labels[0]: int, labels[1]: int, labels[2]: int, labels[3]: int, labels[5]: int})    
df = df.set_index('Id')

#Save file
df.to_csv(DATAFILE)